# Week 47

In [2]:
try:
    import fysisk_biokemi
    print("Already installed")
except ImportError:
    %pip install -q "fysisk_biokemi[colab] @ git+https://github.com/au-mbg/fysisk-biokemi.git"

------------------------------------------------------------------------

## Estimation of binding affinity

#### (a) Train estimation skills

Train your estimation skills using the widget below.

In [3]:
from fysisk_biokemi.widgets.utils.colab import enable_custom_widget_colab
from fysisk_biokemi.widgets import estimate_kd

enable_custom_widget_colab()
estimate_kd()

#### (b) Compare to quadratic

The widget below shows the curves for $\theta$ using both the simple
expression and the quadratic binding expression. Use it to determine how
large \[P\] has to be for them to be notably different.

In [4]:
from fysisk_biokemi.widgets.utils.colab import enable_custom_widget_colab
from fysisk_biokemi.widgets import visualize_simple_vs_quadratic

enable_custom_widget_colab()
visualize_simple_vs_quadratic()

------------------------------------------------------------------------

## The fluorescent protein “mCherry”.

The spectra of many fluorescent proteins can be found at the website:
[www.fpbase.org](https://www.fpbase.org/). Go to FPbase and search for
“mCherry”.

#### (a) Find parameters.

Find the following parameters for the protein

-   Extinction coefficient at absorbance maximum
-   Quantum yield
-   The organism from which it was originally isolated
-   Molecular weight

Save them to seperate variables in the cell below.

In [6]:
... # Your answers here. 

#### (b) Adorbance

What is the absorbance of a 1 µM solution of mCherry at its absorption
maximum?

In [7]:
c = ... # Put the concentration in Molar
l = ... # Path length in cm
A_max = ... # Calculate the adsorbance.
print(f"{A_max = :3.3f}")

#### (c) Extinction coefficient from sequence.

The sequence of the protein is also given. From this determine the
extinction coefficient at 280 nm.

Start by taking the sequence from the website and assigning it to the
variable `sequence` in the cell below.

In [9]:
sequence = ... # Your code here

Now use the sequence to calculate the extinction coefficient, finish the
code below (or take the function you implemented in a previous
exercise!)

In [11]:
# This is "dictionary" with the extinction coefficients of the relevant 
# amino acid residues. Dictionaries are indexed with 'keys', so you can retrieve
# the value for W as: ext_residue["W"].
ext_residue = {"W": 5500, "Y": 1490, "C": 125}

# Write code to calculate the extinction coefficent
...

Ellipsis

#### (d) What is the concentration of a 1 µM solution of mCherry in mg/mL?

In [13]:
conc_molar = ... # Put the concentration in units of M. 
conc_mg_mL = ... # Calculate the concentration in uts of mg/mL.
print(f"{conc_mg_mL = }")

------------------------------------------------------------------------

The excitation and emission spectra can be downloaded as a csv-file by
clicking the download icon as highlighted below

<img width=300 src="https://raw.githubusercontent.com/au-mbg/fysisk-biokemi/refs/heads/main/lessons/figures/week_47/download_spectra.png">

Figure 1: Screenshot of the download button on fpbase

------------------------------------------------------------------------

#### (e) Load the dataset

Use the widget below to load the dataset as a `DataFrame`

In [15]:
from fysisk_biokemi.widgets import DataUploader
from IPython.display import display 
uploader = DataUploader()
uploader.display()

Run the next cell **after** uploading the file

In [16]:
df = uploader.get_dataframe()
display(df)

#### (f) Plot spectra

Make your own plot showing the excitation and emission spectra of
“mCherry” using the above data.

> **Tip**
>
> To plot we use the `matplotlib` package. Plots are generally just
> straight lines connecting points, with enough points we get a smooth
> looking figure.
>
> For example, to plot a line connecting three datapoints
>
> ``` python
> fig, ax = plt.subplots()
>
> x = np.array([1, 2, 3])
> y = np.array([1, 4, 9])
> ax.plot(x, y)
> plt.show()
> ```
>
> Line 1  
> This creates a `fig` and an `ax`, the `ax`-object is the box where our
> plot is created.
>
> Line 3  
> Define the data to put on the x-axis.
>
> Line 5  
> Does the actual plotting
>
> Line 6  
> Shows the figure
>
> There are *many* ways of customizing plots, you will see different
> ones in the exercises, but by no means all of them - if you are
> interested you can find more information on the [matplotlib
> documentation](https://matplotlib.org/stable/users/index).
>
> You don’t have to worry about the `NaN` values in the dataset when
> plotting, matplotlib just skips plotting that line segment.

In [18]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(9, 4))
ax.plot(..., ..., label='Excitation') # Replace ... with your code
... # Replace ... with your code plotting the emission spectrum.
ax.legend()
ax.set_xlabel('Wavelength')
ax.set_ylabel('Extinction & Emission')
plt.show()

#### (g) Stokes shift

What is the Stokes shift of mCherry?

> **Tip**
>
> If you have two arrays `A` and `B` you can find the entry in `A`
> corresponding to the largest value in `B` like this
>
> ``` python
> A_at_B_max = A[np.argmax(B)]
> ```
>
> The `np.argmax` stands for **arg**ument **max**imum meaning that it
> finds the index of the maximum value in a given array. The figure
> below illustrates this
>
> <img width=300 src="https://raw.githubusercontent.com/au-mbg/fysisk-biokemi/refs/heads/main/lessons/figures/week_47/np_argmax.png">
>
> Figure 2: Illustration of argmax

In [20]:
import numpy as np
# Start by extracting the wavelengths from the DataFrame
wavelengths = ...

# Find the indices
ex_max_idx = np.argmax(...)
em_max_idx = ...

# Use the index to find the corresponding wavelengths
lambda_ex = wavelengths[...]
lambda_em = ...

stokes_shift = ...

print(f"{lambda_ex = :d}")
print(f"{lambda_em = :d}")
print(f"{stokes_shift = :d}")

#### (h) Colors

What colors are the light that correspond to the excitation and emission
maxima respectively?

------------------------------------------------------------------------

## Proteins in blood plasma.

In this exercise we will learn how Python is excellent for handling
datasets with many data points and how it can be used to apply the same
procedure to all the data points at once.

A researcher wants to determine the concentration of two proteins in
blood plasma that is suspected to be involved in development of an
autoimmune disease. 500 patients and 500 healthy individuals were
included in the study and absorbance measurements of the two purified
proteins from all blood plasma samples were measured at 280 nm. The
molecular weight and extinction coefficients of the two proteins are
given in the table below.

| Protein | $M_w$ $[\text{kDa}]$ | $\epsilon$ $[\text{M}^{-1}\text{cm}^{-1}]$ |
|-----------------------|---------------------|-----------------------------|
| 1       | 130                  | 180000                                     |
| 2       | 57                   | 80000                                      |

Properties of the two proteins.

#### (a) Load the dataset

Use the widget to load the dataset as a dataframe from the file
**week47_3_protein_blood_plasma.xlsx**

In [22]:
from IPython.display import display 
from fysisk_biokemi.widgets import DataUploader
uploader = DataUploader()
uploader.display()

Run this cell **after** having uploaded the file in the cell above.

In [23]:
df = uploader.get_dataframe()
display(df)

#### (b) Calculate concentrations

Calculate the molar concentration of the two proteins in all samples,
the light path for every measurement is 0.1 cm.

Always a good idea to assign known values to variables

In [25]:
protein_1_ext_coeff = 180000
protein_2_ext_coeff = 80000
l = 0.1

> **Tip**
>
> You can set new columns in a `DataFrame` by just assigning to it
>
> ``` python
> df['new_column'] = [1, 2, 3, ..., 42]
> ```
>
> It can also be set as a computation of a property from another row
>
> ``` python
> df['new_column'] = df['current_column'] / 4
> ```

In [26]:
df['protein1_healthy_molar_conc'] = ... # Calculate for concentration in healthy for protein 1.
... # Your code that updates the data frame with the 3 other new columns.
display(df)

#### (c). Concentrations in mg/mL

Add another set of four columns containing the concentrations in mg/mL.

In [28]:
protein_1_mw = 130 * 10**3
protein_2_mw = 57 * 10**3

In [29]:
df['protein1_healthy_conc'] = ...
df['protein1_patient_conc'] = ...
df['protein2_healthy_conc'] = ...
df['protein2_patient_conc'] = ...

names = ['protein1_healthy_conc', 'protein1_patient_conc', 'protein2_healthy_conc', 'protein2_patient_conc']
display(df[names])

#### (d) Mean concentration

Now that we have the concentrations, calculate the concentration in the
four categories.

> **Tip**
>
> When displaying the dataframe above we used indexed it with `names` as
> `df[names]`. We can do the same to compute something over just the
> four rows.
>
> For example the if we have a `DataFrame` called `example_df`, we can
> calculate the mean over the **rows** as:
>
> ``` python
> df[names].mean(axis=0)
> ```
>
> Here `axis=0` means that we apply the operation over the first axis
> which by convention are the rows. The figure below visualizes this
>
> ![](https://numpy.org/doc/stable/_images/np_matrix_aggregation_row.png)

In [31]:
mean = ...
display(mean)

#### (e) Standard deviation

Calculate the standard deviation

> **Tip**
>
> The standard deviation can be calculated using the `.std`-method that
> works in the same way as the `.mean`-method we used above.

In [33]:
std = ...
display(std)

#### (f) Analyze the results

Consider the following questions

-   By comparing healthy individuals with patients, would you expect
    that any of the two proteins would be involved in disease
    development?
-   What additional information does the standard deviation provide
    besides the average value of the protein concentration?

------------------------------------------------------------------------

## Dialysis experiment

In [35]:
import numpy as np

A dialysis experiment was set up where equal amounts of a protein were
separately dialyzing against buffers containing different concentrations
of a ligand – each measurement was done in triplicate. The average
number of ligands bound per protein molecule, $\bar{n}$ were obtained
from these experiments. The corresponding concentrations of free ligand
and values are given in dataset `week47_4_dialysis_experiment.xlsx`.

#### (a) Load the dataset

In [36]:
from fysisk_biokemi.widgets import DataUploader
from IPython.display import display 
uploader = DataUploader()
uploader.display()

Run the next cell **after** uploading the file

In [37]:
df = uploader.get_dataframe()
display(df)

#### (b) Explain calculation of $\bar{n}$

Explain how the values of $\bar{n}$ is calculated when knowing the
concentrations of ligand inside and outside the dialysis bag, as well as
the total concentration of the protein, \[$\text{P}_{\text{tot}}$\].

#### (c) Molar concentrations

Convert the concentrations of free ligand to SI-units given in M, add it
as a row to the `DataFrame`.

#### (c) Plot the data

In [40]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()

# The 'o' means we plot just the points and don't connect them.
ax.plot(..., ..., 'o') # Replace ... with your code.

ax.set_xlabel('Free ligand [L] (M)', fontsize=16)
ax.set_ylabel(r'$\bar{n}$', fontsize=16)

#### (d) Prepare for fitting

Now we want to fit the data to extract $K_D$ and $\nu_{\text{max}}$, by
using the equation

$$
\nu([L_{\text{free}}]) = \nu_{\text{max}} \frac{[L_{\text{free}}]}{K_D + [L_{\text{free}}]}
$$

To do so we need to implmenet it as a Python function

In [42]:
def nu(L, nu_max, K_D):
    # Replace ... with your code that calculates the above equation. 
    # Be careful with parenthesis!
    result = ... 
    return result

print(f"{nu(1, 1, 1) = }") # Should give 1/2
print(f"{nu(21, 47, 2.5) = }") # Should give 42

#### (e) Actually fitting

> **Important**
>
> Fitting refers to finding the parameters that make an assumed
> functional form best ‘fit’ the data. Programmatically we will use the
> `curve_fit` from the `scipy` package to do so. The signature of this
> function looks like this
>
> ``` python
> curve_fit(function, 
>             x_data, 
>             y_data, 
>             p0=[param_1, param_2, ...])
> ```
>
> The arguments are
>
> -   `function`: A python function where the **first** argument is the
>     independent variable, and other arguments are the parameters of
>     the functions.
> -   `x_data`: The observed values of the independent variable.
> -   `y_data`: The observed values of the dependent variable.
> -   `p0`: Initial guesses for the parameters.
>
> When called `curve_fit` starts by calculating how well the functions
> fits the data with the initial parameters in `p0` and then iteratively
> improves the fit by trying new values for the parameters in an
> intelligent way.
>
> The found parameters will generally depend on `p0` and it is therefore
> necessary to provide a good (or good enough) guess for `p0`.
>
> Generally, the best way to learn more about a function is to read it’s
> documentation and then play around with it. The documentation is in
> this case on the [SciPy
> website](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html).
> You don’t need to read it, unless you want more details.

Finish the code to perform the fitting in the cell below.

In [44]:
from scipy.optimize import curve_fit

# Choose the variables from the dataframe
x = df['Free Ligand [L](M)']
y = df['n-bar']

# Initial guess
K_D_guess = ... # Your initial guess for K_D 
nu_max_guess = ... # Your initial guess for nu_max
p0 = [K_D_guess, nu_max_guess]

# Curve fit
# Replace the four ... with the correct arguments in the correct order.
popt, pcov = curve_fit(..., ..., ..., ...) 

# Print the parameters
nu_max_fit, K_D_fit = popt
print(f"{nu_max_fit = :1.3f} ")
print(f"{K_D_fit = :e}")

Are the parameters you find reasonable? How can you tell if they are
reasonable by looking at the plot you made earlier?

#### (f) Plot with fit

When we have the fitted parameters we can calculate and plot the
function. To do so we make an array of values for the independent
variable and use our function to calculate the dependent variable

In [46]:
# This makes 50 equally spaced points between 0 and the highest concentration x 1.2
L = np.linspace(0, x.max()*1.2, 50) 

# Calculate by plugging L and the found parameters into the function.
nu_calc = ...

Now that we calculated the dependent variable we can plot the fit along
with the data.

------------------------------------------------------------------------

## ADP binding to pyruvate kinase.

The binding of ADP to the enzyme pyruvate kinase was measured by
fluorescence. The enzyme concentration was 4 μM throughout the
titration, and each measurement was done in triplicate. The binding
results were obtained at 310 K and are given in the `.csv`-file
`week47_5_adp_pyruvate.csv`.

#### (a) Load the dataset

As always, use the widget to load the dataset

In [49]:
from fysisk_biokemi.widgets import DataUploader
from IPython.display import display 
uploader = DataUploader()
uploader.display()

Run the next cell **after** uploading the file

In [50]:
df = uploader.get_dataframe()
display(df)

#### (b) Units

The concentrations in the dataset are given in mM, add a new column to
the `DataFrame` with the units given in M.

In [52]:
df['[ADPtot](M)'] = ...
display(df)

#### (c) Free ADP concentration

For each value of $\bar{n}$ calculate the concentration of
\[ADP$_\text{free}$\] from \[ADP$_\text{tot}$\] and \[enzyme\].

In [54]:
enzyme_conc = ...
df['[ADPfree](M)'] = ...
display(df)

#### (d) Make a plot

Make a plot of the free ligand concentration versus $\bar{n}$.

In [56]:
fig, ax = plt.subplots(figsize=(7, 4))

# Write the code to make the plot.
# Add the arguments:  marker='o', linestyle='none' to only plot points and not line segments.
... # Add your code here!

ax.set_xlabel(r'$[\text{ADP}_{\text{free}}]$', fontsize=14)
ax.set_ylabel(r'$\bar{n}$', fontsize=14)

#### (f) Preparing for fitting

To fit we need a implement the function we want to fit the parameters
of, the functional form is

$$
n = n_{\text{max}} \frac{[L]^n}{K_D + [L]^n}
$$

In [58]:
def n_bound(L, n_max, K_D, n_exp):
    return ...

print(f"{n_bound(1, 1, 1, 1) = }") # Should give 1/2
print(f"{n_bound(21, 47, 2.5, 1) = }") # Should give 42
print(f"{n_bound(21, 47, 2.5, 2) = }") # Should give 46.73..

#### (e) Fitting

Finish the code below to create a fit.

In [60]:
from scipy.optimize import curve_fit

# Choose the variables from the dataframe
x = df['[ADPfree](M)']
y = df['nbar']

# Initial guess
K_D_guess = ... # Your initial guess for K_D 
nu_max_guess = ... # Your initial guess for nu_max
n_exp = ... # Your initial guess for the exponent.
p0 = [K_D_guess, nu_max_guess, n_exp]

# Bounds
bounds = (0, np.inf) # We limit the parameters to be positve.

# Curve fit
popt, pcov = curve_fit(n_bound, x, y, p0=p0, bounds=bounds)

# Print the parameters
n_max_fit, K_D_fit, n_exp_fit = popt
print(f"{n_max_fit = :1.3f} ")
print(f"{K_D_fit = :e}")
print(f"{n_exp_fit = :1.3f} ")

Once we’ve obtained the fitted parameters we can plot the fit together
with the data.

In [62]:
L = np.linspace(0, 1.2 * x.max(), 50)
n = n_bound(L, n_max_fit, K_D_fit, 1)

fig, ax = plt.subplots(figsize=(7, 4))
ax.plot(df['[ADPfree](M)'], df['nbar'], 'o', label='Observations')
ax.plot(L, n)
ax.axvline(K_D_fit, color='k', linewidth=0.5, linestyle='--')
ax.axhline(n_max_fit, color='k', linewidth=0.5, linestyle='--')


ax.set_xlabel(r'$[\text{ADP}_{\text{free}}]$', fontsize=14)
ax.set_ylabel(r'$\bar{n}$', fontsize=14)
ax.set_ylim([0, n.max()*1.1])

### Free energy

Calculate the free energy for the association of the ADP-pyruvate kinase
complex assuming
$R = 8.314472 \times 10^{-3} \ \frac{\text{kJ}}{\text{mol} \cdot \text{K}}$
and $T = 310 \ \text{K}$.

> **Tip**
>
> Consider the difference between association and dissociation

Start by defining the two given constants as variables

In [64]:
R = ...
T = ...

And do the calculation

In [66]:
delta_G = ...
print(f"{delta_G = :.3f} kJ/mol")